In [1]:
import pygame
import random
import math
import numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.8.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [23]:
SAFETY_DISTANCE = 20

In [27]:
class Car:
    def __init__(self, position, direction, object_in_front = None):
        self.pos = position
        self.direction = direction
        self.object_in_front = object_in_front
        self.speed = 2/10
        self.max_speed = 4/5
        self.acceleration = 0.02
        self.deceleration = 0
        self.lookahead = self.speed * 20
        self.color = (255, 255, 255)
        self.distance = self.determine_distance()
        
    def euclidean_distance(self, a, b):
        return np.sqrt(np.sum((a - b)**2))
        
    def determine_distance(self):
        return self.euclidean_distance(self.pos,self.object_in_front.pos)
        
    def step(self):
        new_distance = self.determine_distance()
        
        if new_distance < self.lookahead + SAFETY_DISTANCE:
            self.speed = max(self.speed/2, 0)
        else:
            self.speed = min(self.speed+self.acceleration, self.max_speed)
        
        self.distance = new_distance
        self.lookahead = self.speed * 20
        self.pos = self.pos + (self.speed*self.direction)

In [28]:
class Stoplight:
    def __init__(self, position):
        self.pos = position

In [30]:
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Traffic Simulation")
img = pygame.image.load("intersection.png").convert()

#one stoplight
stoplight1 = Stoplight(np.array([400, 300]))
#let's create only one car
queue = []
car1 = Car(np.array([400, 580]), np.array([0, -1]), object_in_front = stoplight1)
queue.append(car1)
car2 = Car(np.array([400, 600]), np.array([0, -1]), object_in_front = car1)
queue.append(car2)

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
            
    screen.fill((100, 100, 100))
    i = 0
    for car in queue:
        pygame.draw.circle(screen, car.color, car.pos, 5)
        car.step()
        if car.pos[1] <= 0:
            car.pos[1] = 600
        if car.speed > 0.00001:
            i += 1
        
    pygame.display.update()
    if i == 0:
        break

In [31]:
pygame.quit()